# Quick Agents Intro

In this notebook we'll see a quick intro to LangChain agents. For a more thorough introduction, check out this notebook.

To use agents we need three main components:

Large Language Models (LLMs)
Tools
The Agents themselves
Let's get started by initializing a LLM:

In [15]:
import inspect

from getpass import getpass
from langchain import OpenAI
from langchain import PromptTemplate 

from langchain.callbacks import get_openai_callback

import tiktoken
from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI



In [16]:
import os
import openai

os.environ["OPENAI_API_TYPE"] = openai.api_type = "azure"
os.environ["OPENAI_API_VERSION"] = openai.api_version = "2022-12-01" #"2023-03-15-preview"
os.environ["OPENAI_API_BASE"] = openai.api_base =  "https://idocopenaigpt.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = openai.api_key = "95776649ac7a4b048c834003fd315264"#os.getenv("AZUREOPENAI_API_KEY")

In [17]:
llm = AzureChatOpenAI(
    deployment_name="chatllm16k",
    openai_api_version="2023-03-15-preview",
    model_name="gpt-3.5-turbo",
    temperature=0,
)

C:\Users\Ramshankar\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.AzureChatOpenAI instead.
  if instance is not None or owner is not None:
C:\Users\Ramshankar\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  if instance is not None or owner is not None:
C:\Users\Ramshankar\anaconda3\Lib\site-packages\langchain_community\chat_models\azure_openai.py:165: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://idocopenaigpt.openai.azure.com/ to https://idocopenaigpt.openai.azure.com/openai.
  warnings.warn(
C:\Users\Ramshanka

Next, initialize a calculator tool using a LLMMathChain:

In [27]:

from langchain.chains import LLMMathChain
from langchain.agents import Tool

llm_math = LLMMathChain(llm=llm)

# initialize the math tool
math_tool = Tool(
    name='Calculator',
    func=llm_math.run,
    description='Useful for when you need to answer questions about math.'
)
# when giving tools to LLM, we must pass as list of tools
tools = [math_tool]

C:\Users\Ramshankar\anaconda3\Lib\site-packages\langchain\chains\llm_math\base.py:57: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [28]:
tools[0].name, tools[0].description

('Calculator', 'Useful for when you need to answer questions about math.')

In [29]:

from langchain.agents import load_tools

tools = load_tools(
    ['llm-math'],
    llm=llm
)
 

In [30]:

tools[0].name, tools[0].description

('Calculator', 'Useful for when you need to answer questions about math.')

Now we use our tools (containing a single tool for now) to initialize an agent:

In [9]:

from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
	agent="zero-shot-react-description",
	tools=tools,
	llm=llm,
	verbose=True,
	max_iterations=3
)
  

In [ ]:
# Let's try asking some questions:



In [10]:
zero_shot_agent("what is (4.5*2.1)^2.2?")




> Entering new AgentExecutor chain...
I need to calculate the value of (4.5*2.1)^2.2.
Action: Calculator
Action Input: (4.5*2.1)^2.2
Observation: Answer: 139.94261298333066
Thought:I now know the final answer
Final Answer: 139.94261298333066

> Finished chain.


{'input': 'what is (4.5*2.1)^2.2?', 'output': '139.94261298333066'}

In [11]:
(4.5*2.1)**2.2

139.94261298333066

Looks good, let's try something else:

In [12]:

zero_shot_agent("if Mary has four apples and Giorgio brings two and a half apple "
                "boxes (apple box contains eight apples), how many apples do we "
                "have?")



> Entering new AgentExecutor chain...
We need to calculate the total number of apples.
Action: Calculator
Action Input: 4 + (2.5 * 8)
Observation: Answer: 24.0
Thought:I now know the final answer
Final Answer: We have 24 apples.

> Finished chain.


{'input': 'if Mary has four apples and Giorgio brings two and a half apple boxes (apple box contains eight apples), how many apples do we have?',
 'output': 'We have 24 apples.'}

But what if we ask a non-math question? Something that a plain and simple LLM should be able to answer easily?

In [13]:

zero_shot_agent("what is the capital of India?")



> Entering new AgentExecutor chain...


ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `I don't need a calculator for this question. I just need to recall the capital of India.
Action: None`

Unfortunately we end up hitting an error. That is because the agent is enforcing the use of a tool (this isn't always the case). A workaround we can implement is to just add another tool that can asnswer this question.

If we'd like the agent to answer this "general knowledge" question. We can, we just need to link it to an LLM tool.

We initialize the tool like so:

In [14]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt = PromptTemplate(
    input_variables=["query"],
    template="{query}"
)

llm_chain = LLMChain(llm=llm, prompt=prompt)

# initialize the LLM tool
llm_tool = Tool(
    name='Language Model',
    func=llm_chain.run,
    description='use this tool for general purpose queries and logic'
)

In [15]:
#Add the new tool to our tools list:
tools.append(llm_tool)

In [16]:

zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3
)
    

In [17]:
zero_shot_agent("what is the capital of India?")



> Entering new AgentExecutor chain...
I can use the Language Model to answer this question.
Action: Language Model
Action Input: "What is the capital of India?"
Observation: The capital of India is New Delhi.
Thought:I now know the final answer.
Final Answer: The capital of India is New Delhi.

> Finished chain.


{'input': 'what is the capital of India?',
 'output': 'The capital of India is New Delhi.'}

In [18]:
zero_shot_agent("what is (4.5*2.1)^2.2?")



> Entering new AgentExecutor chain...
I can use the calculator to solve this math problem.
Action: Calculator
Action Input: (4.5*2.1)^2.2
Observation: Answer: 139.94261298333066
Thought:I now know the final answer
Final Answer: 139.94261298333066

> Finished chain.


{'input': 'what is (4.5*2.1)^2.2?', 'output': '139.94261298333066'}

hello


# Agents 🤖

Agents are like "tools" for LLMs. They allow a LLM to access Google search, perform complex calculations with Python, and even make SQL queries.

In this notebook we'll explore agents and how to use them in LangChain.

We'll start by installing the prerequisite libraries that we'll be using in this example.

In [3]:

# !pip install google-search-results wikipedia sqlalchemy

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32077 sha256=bc486f357d1918462d7d525226d550b3de22468f8b186beb1cc38d95620b1404
  Stored in directory: c:\users\ramshankar\appdata\local\pip\cache\wheels\6e\42\3e\aeb691b02cb7175ec70e2da04b5658d4739d2b41e5f73cd06f
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11707 sha256=96801a8a79f616022bcf1b3ac363fd77048a52bbb18bf4822ac59756b197f72b
  Stored in directory: c:\users\ramshankar\appdata\local\pip\cache\wheels\8f\ab\cb\45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built google-search-results wikipedia


To run this notebook, we will need to use an OpenAI LLM. Here we will setup the LLM we will use for the whole notebook, just input your openai api key when prompted.

In [35]:
from langchain.callbacks import get_openai_callback

def count_tokens(agent, query):
    with get_openai_callback() as cb:
        result = agent(query)
        print(f'Spent a total of {cb.total_tokens} tokens')

    return result

# What is an agent?
__Definition:__ The key behind agents is giving LLM's the possibility of using tools in their workflow. This is where langchain departs from the popular chatgpt implementation and we can start to get a glimpse of what it offers us as builders. Until now, we covered several building blocks in isolation. Let's see them come to life.

The official definition of agents is the following:

Agents use an LLM to determine which actions to take and in what order. An action can either be using a tool and observing its output, or returning to the user.

In this edition we will cover what we may call 'generic' agents which really able to perform many meta tasks. There are other more specific agents that are tuned for different tasks (called 'agent-toolkits'), but we will cover those in a future edition.

Create database
We will use the agents to interact with a small sample database of stocks. We will not dive into the details because this is just a dummy tool we will build for illustrative purposes. Let's create it.

## Create database

We will use the agents to interact with a small sample database of stocks. We will not dive into the details because this is just a dummy tool we will build for illustrative purposes. Let's create it.

In [5]:
from sqlalchemy import MetaData

metadata_obj = MetaData()

In [6]:

from sqlalchemy import Column, Integer, String, Table, Date, Float

stocks = Table(
    "stocks",
    metadata_obj,
    Column("obs_id", Integer, primary_key=True),
    Column("stock_ticker", String(4), nullable=False),
    Column("price", Float, nullable=False),
    Column("date", Date, nullable=False),
)
     

In [7]:
from sqlalchemy import create_engine

engine = create_engine("sqlite:///:memory:")
metadata_obj.create_all(engine)

In [8]:

from datetime import datetime

observations = [
    [1, 'ABC', 200, datetime(2023, 1, 1)],
    [2, 'ABC', 208, datetime(2023, 1, 2)],
    [3, 'ABC', 232, datetime(2023, 1, 3)],
    [4, 'ABC', 225, datetime(2023, 1, 4)],
    [5, 'ABC', 226, datetime(2023, 1, 5)],
    [6, 'XYZ', 810, datetime(2023, 1, 1)],
    [7, 'XYZ', 803, datetime(2023, 1, 2)],
    [8, 'XYZ', 798, datetime(2023, 1, 3)],
    [9, 'XYZ', 795, datetime(2023, 1, 4)],
    [10, 'XYZ', 791, datetime(2023, 1, 5)],
]

In [9]:

from sqlalchemy import insert

def insert_obs(obs):
    stmt = insert(stocks).values(
    obs_id=obs[0],
    stock_ticker=obs[1],
    price=obs[2],
    date=obs[3]
    )

    with engine.begin() as conn:
        conn.execute(stmt)

In [10]:

for obs in observations:
    insert_obs(obs)

We are installing the `langchain_experimental` library here, since the `SQLDatabaseChain` is located there. This might be changed in the future and moved into official langchain library.

In [11]:
# !pip install langchain_experimental

  Obtaining dependency information for langchain_experimental from https://files.pythonhosted.org/packages/1a/55/fb5e193c7c7eb00e9e080d2ea371f708e03e1f181e260e7e40010ab9142b/langchain_experimental-0.0.48-py3-none-any.whl.metadata
  Obtaining dependency information for langchain<0.2,>=0.1 from https://files.pythonhosted.org/packages/23/98/c70fac0f1b3193ced86013b563119c27c68ac26b684815f407555224108d/langchain-0.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for langchain-core<0.2.0,>=0.1.7 from https://files.pythonhosted.org/packages/61/45/79001a591b55493a204a025bc164092a312714256e1748273110a2450a22/langchain_core-0.1.9-py3-none-any.whl.metadata
  Obtaining dependency information for langchain-community<0.1,>=0.0.9 from https://files.pythonhosted.org/packages/82/f5/9ae4e971ce553e20aae597df39806ed8d862e8536bd80a11497bebe1262a/langchain_community-0.0.11-py3-none-any.whl.metadata
  Obtaining dependency information for langsmith<0.1.0,>=0.0.77 from https://files.pythonhoste

In [18]:
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

db = SQLDatabase(engine)
sql_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)

C:\Users\Ramshankar\anaconda3\Lib\site-packages\langchain_experimental\sql\base.py:76: UserWarning: Directly instantiating an SQLDatabaseChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


# Agent types

In this section we will review several agents and see how they 'think' and what they can do.

Using one of langchain's pre-built agents involves three variables:

* defining the tools or the toolkit
* defining the llm
* defining the agent type

This is all really easy to do in langchain, as we will see in the following example.

# Agent type #1: Zero Shot React

In this first example we will use slightly different type of agent - SQL Agent which can be instantiated with it's own method `create_sql_agent`. Other agents will be instantiated in more generic way as we will see below in other examples.

This method uses toolkit instead of simple list of `tools`. You can read more about them in the documentation. For this use case, we will use `SQLDatabaseToolkit`.

As the name suggests, we will use this agent to perform 'zero shot' tasks on the input. That means that we will not have several, interdependent interactions but only one. In other words, this agent will have no memory.

Now we are ready to initialize the agent! We will use `verbose` in `True` so we can see what is our agent's 'thinking' process.

__Important Note__: When interacting with agents it is really important to set the `max_iterations` parameters because agents can get stuck in infinite loops that consume plenty of tokens. The default value is 15 to allow for many tools and complex reasoning but for most applications you should keep it much lower.

In [59]:

from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType

agent_executor = create_sql_agent(
    llm=llm,
    toolkit=SQLDatabaseToolkit(db=db, llm=llm),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    max_iterations=3,
    handle_parsing_errors=True
)
 

C:\Users\Ramshankar\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The class `ZeroShotAgent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use create_react_agent instead.
  if instance is not None or owner is not None:
C:\Users\Ramshankar\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The class `Agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  if instance is not None or owner is not None:


 Let's see our newly created agent in action! We will ask it a question that involves a math operation over the stock prices.

In [60]:

result = count_tokens(
    agent_executor,
    "What is the multiplication of the ratio between stock " +
    "prices for 'ABC' and 'XYZ' in January 3rd and the ratio " +
    "between the same stock prices in January the 4th?"
)

C:\Users\Ramshankar\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  if instance is not None or owner is not None:
C:\Users\Ramshankar\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  if instance is not None or owner is not None:




> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: stocks
Thought:

C:\Users\Ramshankar\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  if instance is not None or owner is not None:


The 'stocks' table seems relevant. I should query the schema of this table.
Action: sql_db_schema
Action Input: stocks
Observation: 
CREATE TABLE stocks (
	obs_id INTEGER NOT NULL, 
	stock_ticker VARCHAR(4) NOT NULL, 
	price FLOAT NOT NULL, 
	date DATE NOT NULL, 
	PRIMARY KEY (obs_id)
)

/*
3 rows from stocks table:
obs_id	stock_ticker	price	date
1	ABC	200.0	2023-01-01
2	ABC	208.0	2023-01-02
3	ABC	232.0	2023-01-03
*/
Thought:

C:\Users\Ramshankar\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  if instance is not None or owner is not None:


I can query the 'stocks' table to get the stock prices for 'ABC' on January 3rd and January 4th.
Action: sql_db_query
Action Input: SELECT price FROM stocks WHERE stock_ticker = 'ABC' AND date IN ('2023-01-03', '2023-01-04')
Observation: [(232.0,), (225.0,)]
Thought:

> Finished chain.
Spent a total of 2060 tokens


In [61]:
print(agent_executor.agent.llm_chain.prompt.template)

You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct sqlite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 10 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "I don't k

__The question we must ask ourselves here is: how are agents different than chains?__

If we look at the agent's logic and the prompt we have just printed we will see some clear differences. First, we have the tools which are included in the prompt. Second we have a thought process which was before was immediate in chains but now involves a 'thought', 'action', 'action input', 'observation' sequence. What is this all about?

Suffice it to say for now that the __LLM now has the ability to 'reason' on how to best use tools__ to solve our query and can combine them in intelligent ways with just a brief description of each of them. If you want to learn more about this paradigm (MRKL) in detail, please refer to this paper.

Finally, let's pay attention to the `'agent_scratchpad'`. What is that? Well, that is where we will be appending every thought or action that the agent has already performed. In this way, at each point in time, the agent will know what it has found out and will be able to continue its thought process. In other words, after using a tool it adds its thoughts and observations to the scratchpad and picks up from there.

# Agent type #2: Conversational React

The zero shot agent is really interesting but, as we said before, it has no memory. What if we want an assistant that remembers things we have talked about and can also reason about them and use tools? For that we have the conversational react agent.

We will use the math tool in this example and load it as below:

In [62]:

from langchain.agents import load_tools

tools = load_tools(
    ["llm-math"],
    llm=llm
)
   

In [72]:
# from langchain.prompts import PromptTemplate
# from langchain.chains import LLMChain

# prompt = PromptTemplate(
#     input_variables=["query"],
#     template="{query}"
# )

# llm_chain = LLMChain(llm=llm, prompt=prompt)

# # initialize the LLM tool
# llm_tool = Tool(
#     name='Language Model',
#     func=llm_chain.run,
#     description='use this tool for general purpose queries and logic'
# )

In [81]:
# #Add the new tool to our tools list:
# tools.append(llm_tool)

The memory type being used here is a simple buffer memory to allow us to remember previous steps in the reasoning chain. For more information on memory, please refer to the 3rd chapter of this series.

In [83]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history")

In [84]:
from langchain.agents import initialize_agent

conversational_agent = initialize_agent(
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    memory=memory,
    handle_parsing_errors=True
)

C:\Users\Ramshankar\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  if instance is not None or owner is not None:
C:\Users\Ramshankar\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The class `ConversationalAgent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use create_react_agent instead.
  if instance is not None or owner is not None:
C:\Users\Ramshankar\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The class `Agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. inst

In [86]:


# result = count_tokens(conversational_agent,
#     "What is the simple interest of an amount of $10,000 growing at 2% annually for 5 years?")
  

In [ ]:

print(conversational_agent.agent.llm_chain.prompt.template)

In [44]:

result = count_tokens(
    conversational_agent,
    "If we start with $15,000 instead and follow the same 8% annual growth for 5 years with compound interest, how much more would we have compared to the previous scenario?"
)
  

C:\Users\Ramshankar\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  if instance is not None or owner is not None:
C:\Users\Ramshankar\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  if instance is not None or owner is not None:




> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: Initial amount = $15,000, Annual growth rate = 8%, Number of years = 5

C:\Users\Ramshankar\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  if instance is not None or owner is not None:
C:\Users\Ramshankar\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  if instance is not None or owner is not None:
C:\Users\Ramshankar\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  if instance is not None or owner is not None:



Observation: Answer: 22039.92115200001
Thought:

C:\Users\Ramshankar\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  if instance is not None or owner is not None:


Could not parse LLM output: `Do I need to use a tool? No`
Observation: Invalid or incomplete response
Thought:

C:\Users\Ramshankar\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  if instance is not None or owner is not None:


Could not parse LLM output: `Do I need to use a tool? No`
Observation: Invalid or incomplete response
Thought:

> Finished chain.
Spent a total of 1687 tokens


# Agent type #3: React Docstore

This type of agent is similar to the ones we have seen so far but it includes the interaction with a docstore. It will have two and only two tools at its disposal: 'Search' and 'Lookup'.

With 'Search' it will bring up a relevant article and with 'Lookup' the agent will find the right piece of information in the article. This is probably easiest to see in an example:

In [46]:

from langchain import Wikipedia
from langchain.agents.react.base import DocstoreExplorer, Tool

docstore=DocstoreExplorer(Wikipedia())
tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description='search wikipedia'
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description='lookup a term in wikipedia'
    )
]

C:\Users\Ramshankar\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The class `DocstoreExplorer` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0
  if instance is not None or owner is not None:


In [49]:
docstore_agent = initialize_agent(
    tools,
    llm,
    agent="react-docstore",
    verbose=True,
    max_iterations=3,
    handle_parsing_errors=True
)

C:\Users\Ramshankar\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  if instance is not None or owner is not None:
C:\Users\Ramshankar\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The class `ReActDocstoreAgent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0
  if instance is not None or owner is not None:
C:\Users\Ramshankar\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The class `Agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  if instance is not None or 

In [50]:
count_tokens(docstore_agent, "What were Archimedes' last words?")

C:\Users\Ramshankar\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  if instance is not None or owner is not None:
C:\Users\Ramshankar\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  if instance is not None or owner is not None:




> Entering new AgentExecutor chain...
Thought: I need to search for Archimedes' last words and find out what they were.
Action: Search[Archimedes' last words]
Observation: A person's last words, their final articulated words stated prior to death or as death approaches, are often recorded because of the decedent's fame, but sometimes because of interest in the statement itself. (People dying of illness are frequently inarticulate at the end, and in such cases their actual last utterances may not be recorded or considered very important.) Last words may be recorded accurately, or, for a variety of reasons, may not. Reasons can include simple error or deliberate intent. Even if reported wrongly, putative last words can constitute an important part of the perceived historical records or demonstration of cultural attitudes toward death at the time.Charles Darwin, for example, was reported to have disavowed his theory of evolution in favor of traditional religious faith at his death. This

C:\Users\Ramshankar\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  if instance is not None or owner is not None:


The search results do not provide any information about Archimedes' last words. I may need to search specifically for Archimedes' last words.
Action: Search[Archimedes' last words]
Observation: A person's last words, their final articulated words stated prior to death or as death approaches, are often recorded because of the decedent's fame, but sometimes because of interest in the statement itself. (People dying of illness are frequently inarticulate at the end, and in such cases their actual last utterances may not be recorded or considered very important.) Last words may be recorded accurately, or, for a variety of reasons, may not. Reasons can include simple error or deliberate intent. Even if reported wrongly, putative last words can constitute an important part of the perceived historical records or demonstration of cultural attitudes toward death at the time.Charles Darwin, for example, was reported to have disavowed his theory of evolution in favor of traditional religious fait

C:\Users\Ramshankar\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  if instance is not None or owner is not None:


Could not parse LLM Output: The search results do not provide any information about Archimedes' last words. It seems that his last words may not have been recorded or are not widely known.
Observation: Invalid or incomplete response
Thought:

> Finished chain.
Spent a total of 5274 tokens


{'input': "What were Archimedes' last words?",
 'output': 'Agent stopped due to iteration limit or time limit.'}

We will not print the prompt here because it is too large, but you can see it yourself if you want to (we know how to already).

In short, it contains several examples of the Question > Thought > Action > Observation loop, that include the Search and Lookup tools.

If you want to learn more about this approach this is the paper for ReAct

# Agent type #4: Self Ask with Search

This is the first-choice agent to use when using LLM's to extract information with a search engine. The agent will ask follow-up questions and use the search functionality to get intermediate answers that help it get to a final answer.

In [57]:

from langchain import OpenAI, SerpAPIWrapper
from langchain.agents import initialize_agent, Tool

search = SerpAPIWrapper(serpapi_api_key='api_key')
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description='google search'
    )
]

self_ask_with_search = initialize_agent(tools, llm, agent="self-ask-with-search", verbose=True)

C:\Users\Ramshankar\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  if instance is not None or owner is not None:
C:\Users\Ramshankar\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The class `SelfAskWithSearchAgent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use create_self_ask_with_search instead.
  if instance is not None or owner is not None:
C:\Users\Ramshankar\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The class `Agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agen

We will not interact with this agent because for that we would need a serpapi key. However, by checking out the prompt we can see a few examples of how it works:

In [58]:
print(self_ask_with_search.agent.llm_chain.prompt.template)

Question: Who lived longer, Muhammad Ali or Alan Turing?
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali

Question: When was the founder of craigslist born?
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952

Question: Who was the maternal grandfather of George Washington?
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washin

As we can see, the prompt is basically a series of many examples to show the LLM how to ask follow up questions to a search tool until it can get to the final answer.


Wrapping up
And that all for agents! There's many other things you can do with agents, just to name a few:

* Create your own custom agent
* Use them with many other tools (even custom ones)
* Trace every call an agent makes through a convinient UI interface